In [1]:
#!/usr/bin/env python

import tensorflow as tf
import numpy as np
import input_data


def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01), name=name)

# This network is the same as the previous one except with an extra hidden layer + dropout
def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden):
    # Add layer name scopes for better graph visualization
    with tf.name_scope("layer1") as scope:
        X = tf.nn.dropout(X, p_keep_input)
        h = tf.nn.relu(tf.matmul(X, w_h))
    with tf.name_scope("layer2") as scope:
        h = tf.nn.dropout(h, p_keep_hidden)
        h2 = tf.nn.relu(tf.matmul(h, w_h2))
    with tf.name_scope("layer3") as scope:
        h2 = tf.nn.dropout(h2, p_keep_hidden)
        return tf.matmul(h2, w_o)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

X = tf.placeholder("float", [None, 784], name="X")
Y = tf.placeholder("float", [None, 10], name="Y")

w_h = init_weights([784, 625], "w_h")
w_h2 = init_weights([625, 625], "w_h2")
w_o = init_weights([625, 10], "w_o")

# Add histogram summaries for weights
tf.histogram_summary("w_h_summ", w_h)
tf.histogram_summary("w_h2_summ", w_h2)
tf.histogram_summary("w_o_summ", w_o)

p_keep_input = tf.placeholder("float", name="p_keep_input")
p_keep_hidden = tf.placeholder("float", name="p_keep_hidden")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
    train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    # Add scalar summary for cost
    tf.scalar_summary("cost", cost)

with tf.name_scope("accuracy") as scope:
    correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(py_x, 1)) # Count correct predictions
    acc_op = tf.reduce_mean(tf.cast(correct_pred, "float")) # Cast boolean to float to average
    # Add scalar summary for accuracy
    tf.scalar_summary("accuracy", acc_op)

with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs", sess.graph)
    merged = tf.merge_all_summaries()

    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(10):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})
        summary, acc = sess.run([merged, acc_op], feed_dict={X: teX, Y: teY,
                                          p_keep_input: 1.0, p_keep_hidden: 1.0})
        writer.add_summary(summary, i)  # Write summary
        print(i, acc)                   # Report the accuracy

Extracting MNIST_data/train-images-idx3-ubyte.gz


/Users/act65/anaconda/lib/python3.5/gzip.py:274: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return self._buffer.read(size)
/Users/act65/Documents/Jupyter/Tensorflow/input_data.py:38: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0 0.9352
1 0.9617
2 0.9721
3 0.9747
4 0.9717
5 0.9764
6 0.9772
7 0.977
8 0.9762
9 0.9792


In [ ]:
!tensorboard --logdir=./logs/nn_logs

Starting TensorBoard  on port 6006
(You can navigate to http://0.0.0.0:6006)
127.0.0.1 - - [18/Apr/2016 10:58:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2016 10:58:07] code 404, message Not Found
127.0.0.1 - - [18/Apr/2016 10:58:07] "GET /lib/css/global.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Apr/2016 10:58:07] "GET /external/lodash/lodash.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2016 10:58:07] "GET /external/d3/d3.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2016 10:58:07] "GET /external/graphlib/dist/graphlib.core.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2016 10:58:07] "GET /external/plottable/plottable.css HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2016 10:58:07] "GET /external/plottable/plottable.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2016 10:58:07] "GET /external/dagre/dist/dagre.core.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2016 10:58:07] "GET /external/polymer/polymer.html HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2016 10:58:07] "GET /external/webcomponentsjs/webcom